# HW 6


## Part I
Dataset Source: https://www.kaggle.com/datasets/dileep070/heart-disease-prediction-using-logistic-regression

We will predict whether the person is having diabetics or not with predictors: glucose, pregnancies, and blood pressure.

In [19]:
import pandas as pd
import pymc as pm

data_path = '/content/sample_data/diabetes2.csv'
df = pd.read_csv(data_path)
df = df.dropna()

X = df[['Glucose', 'Pregnancies', 'BloodPressure']]
y = df['Outcome']

with pm.Model() as logistic_model:
    intercept = pm.Normal('Intercept', mu=0, sigma=10)
    beta = pm.Normal('beta', mu=0, sigma=10, shape=X.shape[1])

    mu = intercept + pm.math.dot(X, beta)
    observed = pm.Bernoulli('obs', logit_p=mu, observed=y)

    trace = pm.sample(1000, tune=1000, target_accept=0.95, return_inferencedata=True)
pm.summary(trace)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
Intercept,-2.819,2.771,-6.165,-0.071,1.939,1.636,3.0,103.0,2.23
beta[0],0.293,0.255,0.034,0.547,0.180,0.152,3.0,161.0,2.23
beta[1],0.200,0.075,0.099,0.273,0.052,0.044,3.0,88.0,2.23
beta[2],-0.486,0.482,-0.968,0.001,0.340,0.287,3.0,2.0,2.24


In [20]:
with pm.Model() as logistic_model:
    betas = pm.Normal('betas', mu=0, sigma=10, shape=X.shape[1])
    intercept = pm.Normal('Intercept', mu=0, sigma=10)

    logits = intercept + pm.math.dot(X, betas)
    observed_outcomes = pm.Bernoulli('y', logit_p=logits, observed=y)

    trace = pm.sample(draws=1000, tune=1000, target_accept=0.95)

pm.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
betas[0],0.141,0.103,0.034,0.244,0.072,0.061,3.0,173.0,2.23
betas[1],0.066,0.064,0.005,0.159,0.043,0.036,3.0,2.0,2.24
betas[2],-0.419,0.415,-0.833,0.001,0.292,0.247,3.0,2.0,2.23
Intercept,-2.815,2.807,-6.255,-0.031,1.964,1.656,3.0,65.0,2.23


Q3.
The estimated mean of the intercept is -2.815 with a standard deviation of 2.807. We choose a Normal prior centered around -2.8 with a smaller standard deviation of 2 to make it more informative. Similarly, for the betas, we let the mu be the mean of the estimated betas, and sigma be their standard deviations.

In [21]:
with pm.Model() as logistic_model:

    betas = pm.Normal('betas', mu=[0.14, 0.07, -0.42], sigma=[0.05, 0.03, 0.2], shape=X.shape[1])
    intercept = pm.Normal('Intercept', mu=-2.8, sigma=2)

    logits = intercept + pm.math.dot(X, betas)
    observed_outcomes = pm.Bernoulli('y', logit_p=logits, observed=y)

    trace = pm.sample(draws=1000, tune=1000, target_accept=0.95, return_inferencedata=True)

pm.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
betas[0],0.037,0.003,0.031,0.043,0.000,0.000,991.0,879.0,1.0
betas[1],0.102,0.020,0.064,0.137,0.001,0.000,1327.0,958.0,1.0
betas[2],-0.005,0.004,-0.013,0.004,0.000,0.000,1247.0,1086.0,1.0
Intercept,-5.371,0.466,-6.181,-4.430,0.017,0.012,786.0,813.0,1.0


## Part II
For a linear regression model $y = X\beta + \epsilon$ with $\epsilon \sim N(0, \sigma^2I)$, the likelihood function is:

$$
p(y|X,\beta) \propto \exp\left(-\frac{1}{2\sigma^2}||y-X\beta||_2^2\right)
$$

For $\sigma=1$, this simplifies to:

$$
p(y|X,\beta) \propto \exp\left(-\frac{1}{2}||y-X\beta||_2^2\right)
$$

### Normal Prior (Ridge Regression)
The normal (Gaussian) prior for coefficients $\beta$ with mean 0 and variance $\tau^2$ (where $\lambda = 1/\tau^2$) is:

$$
p(\beta) \propto \exp\left(-\frac{\lambda}{2}\sum_{j=1}^n \beta_j^2\right)
$$

The log posterior is, therefore:

$$
\log(p(\beta|X,y)) \propto -\frac{1}{2}||y-X\beta||_2^2 - \frac{\lambda}{2}\sum_{j=1}^n \beta_j^2
$$

This matches the form of ridge regression:

$$
\frac{1}{2}||y-X\beta||_2^2 + \lambda ||\beta||_2^2
$$

### Laplace Prior (Lasso Regression)
The Laplace prior for coefficients $\beta$ with mean 0 and scale parameter $b$ (where $\lambda = 1/b$) is:

$$
p(\beta) \propto \exp\left(-\lambda\sum_{j=1}^n |\beta_j|\right)
$$

The log posterior is thus:

$$
\log(p(\beta|X,y)) \propto -\frac{1}{2}||y-X\beta||_2^2 - \lambda\sum_{j=1}^n |\beta_j|
$$

Matching the form of lasso regression:

$$
\frac{1}{2}||y-X\beta||_2^2 + \lambda ||\beta||_1
$$

## Part III
Same dataset

In [22]:
# X and y are already defined and preprocessed in part I

nu = 4
w = 1

with pm.Model() as robust_regression_model:
    beta = pm.Normal('beta', mu=0, sigma=10, shape=X.shape[1])
    intercept = pm.Normal('Intercept', mu=0, sigma=20)

    mu = pm.math.dot(X, beta) + intercept
    lam = pm.Gamma('lam', alpha=nu/2, beta=nu/2, shape=y.shape[0])

    y_obs = pm.Normal('y_obs', mu=mu, sigma=pm.math.sqrt(1/lam), observed=y)

    trace = pm.sample(1000, tune=1000, target_accept=0.95)
pm.summary(trace)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta[0],0.007,0.001,0.005,0.009,0.000,0.000,1456.0,1440.0,1.00
beta[1],0.024,0.010,0.004,0.043,0.000,0.000,2036.0,1555.0,1.01
beta[2],-0.001,0.002,-0.004,0.003,0.000,0.000,1686.0,1597.0,1.00
Intercept,-0.567,0.166,-0.853,-0.239,0.005,0.004,1127.0,1263.0,1.00
lam[0],1.208,0.791,0.075,2.613,0.015,0.012,2226.0,1002.0,1.00
...,...,...,...,...,...,...,...,...,...
lam[763],1.205,0.762,0.111,2.644,0.017,0.012,1764.0,1141.0,1.00
lam[764],1.219,0.759,0.117,2.572,0.014,0.011,2334.0,1243.0,1.00
lam[765],1.215,0.760,0.119,2.576,0.013,0.010,2483.0,988.0,1.00
lam[766],1.132,0.695,0.061,2.352,0.012,0.009,2658.0,1150.0,1.00


In [23]:
import numpy as np

lam_means = np.mean(trace.posterior['lam'], axis=(0, 1))
threshold = np.quantile(lam_means, 0.05)
outlier_indices = np.where(lam_means < threshold)[0]

outlier_indices


array([  6,  19,  38,  66,  70, 109, 124, 125, 128, 129, 197, 212, 218,
       228, 255, 258, 260, 291, 327, 328, 349, 400, 429, 448, 476, 489,
       502, 540, 542, 549, 577, 622, 659, 660, 670, 709, 719, 739, 744])